<!DOCTYPE html>
<html>
<head>
    <style>
        .center-image {
            display: block;
            margin: 0 auto;
        }
    </style>
</head>
<body>
    <img src="https://cdn.winsightmedia.com/platform/files/public/2018-11/background/mondelez_logo_0_1541177762.png?VersionId=fMbF58AQDNx_lQdhQZyrWhOoW8R6Lnrx" alt="Mondelez" class="center-image">
</body>
</html>

En este notebook se llevará a cabo la limpieza y unificación de las distintas bases de datos proporcionadas por la firma Randstad.

Objetivo: Se necesita crear un tablero de power bi que proporcione información general derivada del proyecto "Censo".

Problemtica:

1) Las bases deben ser unificadas en una misma base para tener la informacion en una misma tabla 
2) las bases son cargadas por censistas. es decir la carga esta sujeta a errores humanos. 


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Configuro pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

## **CONTEXTO Y CARGA DE DATOS:**
Primero se cargarán las distintas bases disponibles para la obtención de la información requerida:

1. **Reporte de Observaciones:** Este reporte contiene la información de instancias particulares del censo, que incluye locales cerrados, locales que rechazaron la encuesta y locales que brindaron poca información.

2. **Reporte de Tareas:** Este reporte contiene la información de los locales denominados "Canales de Comercialización Core" que accedieron a la encuesta y las preguntas correspondiente a cada canal ded comercializacion particular.

3. **Reporte de Entrada y Salida Manual:** Este reporte contiene la información de todos los locales, incluyendo tanto los denominados "Canales de Comercialización Core" como aquellos que no pertenecen a esta categoría. Además, incluye registros únicos que ya se encuentran en el reporte de Observaciones y en el reporte de Tareas.

In [2]:
# carga de datos
df_observaciones = pd.read_excel("Reporte de Observaciones 010524 - 310524.xlsx")
df_tareas = pd.read_excel("Reportes de tareas completadas.xlsx")
df_entrada_salida = pd.read_excel("Reporte de Visitas - Marcas manuales.xlsx")

In [3]:
# Convertir las columnas de identificadores a conjuntos para verificar la inclusión
set_tareas = set(df_tareas['Identificador del punto de interés'])
set_entrada_salida = set(df_entrada_salida['Identificador de Punto de Interés'])

# Verificar si todos los identificadores de df_tareas están en df_entrada_salida
todos_en_entrada_salida = set_tareas.issubset(set_entrada_salida)
# Convertir las columnas de identificadores a conjuntos para verificar la inclusión
set_observaciones = set(df_observaciones['Identificador Punto de interés'])

# Verificar si todos los identificadores de df_observaciones están en df_entrada_salida
todos_en_entrada_salida_obs = set_observaciones.issubset(set_entrada_salida)
# Verificar si todos los identificadores de df_observaciones están en df_tareas
todos_en_tareas_obs = set_observaciones.issubset(set_tareas)

print(f"¿Todos los identificadores de df_observaciones están en df_tareas?: {todos_en_tareas_obs}")
print(f"¿Todos los identificadores de df_observaciones están en df_entrada_salida?: {todos_en_entrada_salida_obs}")
print(f"¿Todos los identificadores de df_tareas están en df_entrada_salida?: {todos_en_entrada_salida}")


¿Todos los identificadores de df_observaciones están en df_tareas?: False
¿Todos los identificadores de df_observaciones están en df_entrada_salida?: True
¿Todos los identificadores de df_tareas están en df_entrada_salida?: True


In [4]:
df_tareas['Canal'].unique()

array(['Maxi Kiosco', 'Autoservicio / Chino', 'Kiosco Ventana', 'Almacén',
       'Verdulería', 'Dietetíca', 'Bar', 'Golsinería', 'Fiambrería'],
      dtype=object)

In [5]:
# Defino los valores a filtrar en la columna 'Canal'
valores_filtrar = ['Fiambrería', 'Verdulería', 'Dietetíca', 'Bar']

# Filtrar el DataFrame
df_filtrado = df_tareas[df_tareas['Canal'].isin(valores_filtrar)]
df_filtrado

,Fecha de inicio,Hora inicio,Coordenadas inicio,Fecha completado,Hora completado,Coordenadas fin,Identificador de persona de interés,Nombre de persona de interés,IMEI del teléfono,Número de teléfono,Identificador del punto de interés,Nombre del punto de interés,Jerarquia 1 del punto de interés,Jerarquia 2 del punto de interés,Departamento del punto de interés,Dirección del punto de interés,Nombre contacto en punto de interés,Teléfono contacto en punto de interés,Provincia,Departamentos,Localidad,Tipo de Punto de Venta,Número de Polígono,Preguntas,Razón Social,CUIT,Papeles?,Canal,Cantidad de Cajas,Cantidad de Góndolas,Tiene Verdulería,Tiene Fiambrería,Heladeras con lacteos,Heladeras con bebidas,Estado de Inmueble,Cantidad de heladeras,Cantidad de Exhibidores de primeras marcas (Lays/Gaseosas/Cervezas),Tiene surtido de vinos,Tiene Heladera de lácteos,Estado de Inmueble2,Cantidad de Heladeras3,Estado del inmueble,Tiene gondola,"En caso de que si tenga gondola, cuan llena esta?",Tiene exhibidor de chicles/caramelos,"En caso de tener exhibidor de chicles/caramelos, cuan lleno esta?",No tiene servicios adicionales,Fotocopias,RapiPago,Tramites,SUBE,Cantidad de Heladeras visibles,Cuan llena estan las estanterias,Estado de inmueble4,Comercializa chocolates,Comercializa productos de kiosco,Modalidad de Venta,Tabletas,Bombones,Alfajores,Chicles,Caramelos,Galletas,Estado del inmueble5,Cantidad de góndolas/estanterías,Llenado de estanterías,Cantidad de heladeras6,Estado de Inmueble7,Trabaja con?,Distribuidor,Frecuencia de Visita,Frecuencia de entrega,Se acerca un repositor de Mondelez?,Se acerca un repositor de la competencia?,"En caso de que sea otro, indicar con quien",Aplicaciones B2B,Mondelez,Arcor,BEES,Serenisima,Massalin,Arcor8,Georgalos,Quilmes,Pepsi,Coca Cola,Cigarrillos,Mondelez9,Observaciones
15,2024-05-02 10:53:54.647,2024-05-02 10:53:54.647,"-34.60000502031288500000,-58.37801836431027000000",2024-05-02 10:54:14.217,2024-05-02 10:54:14.217,"-34.60004210000000000000,-58.37797360000000000000",TA-820118,Tomás Aquilino,240229195109558,1159111811,AU9-135720,Autoservicio viamonte 789,Verdulería,NaN,CABA,Viamonte 789,NaN,NaN,Buenos Aires,Capital,Microcentro,Verdulería,NaN,NaN,NaN,NaN,No,Verdulería,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,2024-05-02 16:25:42.243,2024-05-02 16:25:42.243,"-34.60519020827237600000,-58.37931923568249000000",2024-05-02 16:29:19.413,2024-05-02 16:29:19.413,"-34.60471810000000000000,-58.37921690000000000000",RZ-623877,Ricardo Zorkendorfer,240229194711771,1159111825,CO -291303,Coquito 's,Dietética,NaN,CABA,Suipacha 299,NaN,NaN,Buenos Aires,Capital,Microcentro,Dietética,NaN,NaN,Healthy market,3.071721e+10,Si,Dietetíca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 o más,75%,2,Ordenado y Limpio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN
107,2024-05-03 13:20:10.773,2024-05-03 13:20:10.773,"-34.60793183282597000000,-58.37499015033244400000",2024-05-03 13:22:30.833,2024-05-03 13:22:30.833,"-34.60782490000000000000,-58.37496470000000000000",RZ-623877,Ricardo Zorkendorfer,240229194711771,1159111825,VIR-108684,Vitalcer,Dietética,NaN,CABA,Florida 12,NaN,NaN,Buenos Aires,Capital,Microcentro,Dietética,NaN,NaN,Papamax SA,3.071650e+10,Si,Dietetíca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 o más,50%,3 o más,Ordenado y Limpio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN
128,2024-05-03 16:02:41.447,2024-05-03 16:02:41.447,"-34.59957946121321000000,-58.39349236339331000000",2024-05-03 16:02:50.023,2024-05-03 16:02:50.023,"-34.59965120000000000000,-58

## **ELIMINACION DE COLUMNAS**
A continuación, se procederá a eliminar las columnas que no aportan información necesaria, en particular de los tres reportes el unico reporte que nos proporciona una opcion de si queremos o no una columna es el reporte de tareas. para la creacion de la base se tomo como desicion la convencion de tomar todas las columnas y trabajar la limpieza eliminando desde el mismo procesamiento con python para de esta forma mejorar la automatizacion de la limpieza. para la creación de la base de datos. Las columnas que se eliminarán de las bases de datos son las siguientes:

**Reporte de Observaciones:**
- Imagen 1
- Imagen 2
- Imagen 3
- Persona de interés

Estas columnas se utilizan para constatar observaciones de locales cerrados, pero no son útiles para el propósito de este notebook.

**Reporte de Entrada y Salida:**
- IMEI
- Jerarquía nivel 2
- Tiempo transcurrido
- Identificador Punto de interés salida
- Nombre Punto de interés salida
- Foto de entrada
- Foto de salida
- Nombre
- Dirección2
- Nombre de contacto
- Teléfono de contacto
- Departamento3
- Agrupaciones
- Jerarquía nivel 14
- Jerarquía nivel 25
- E-mails
- Imagen
- Ventana horaria

**Reporte de Tareas:**
- Preguntas
- IMEI del teléfono
- Jerarquia 2 del punto de interés
- Departamento del punto de interés
- Dirección del punto de interés
- Nombre contacto en punto de interés
- Teléfono contacto en punto de interés

In [6]:
# Elimino columnas no necesarias del DataFrame de observaciones
columnas_observaciones = ['Imagen 1', 'Imagen 2', 'Imagen 3','Persona de interés']
df_observaciones = df_observaciones.drop(columns=columnas_observaciones)

# Elimino columnas no necesarias del DataFrame de entrada y salida
columnas_entrada_salida = [
    'IMEI', 'Jerarquía nivel 25', 'Jerarquía nivel 2', 'Tiempo transcurrido',
    'Identificador Punto de interés salida', 'Nombre Punto de interés salida',
    'Foto de entrada', 'Foto de salida', 'Nombre', 'Dirección', 'Nombre de contacto',
    'Teléfono de contacto', 'Departamento', 'Agrupaciones',
    'Jerarquía nivel 14', 'E-mails', 'Imagen', 'Ventana horaria'
]
df_entrada_salida = df_entrada_salida.drop(columns=columnas_entrada_salida)

# Elimino columnas no necesarias del DataFrame de tareas
columnas_tareas = ['Preguntas','IMEI del teléfono', 'Jerarquia 2 del punto de interés',
                   'Departamento del punto de interés','Dirección del punto de interés'
                   ,'Nombre contacto en punto de interés','Teléfono contacto en punto de interés']
df_tareas = df_tareas.drop(columns=columnas_tareas)

En nuestros reportes, encontramos tanto información duplicada como complementaria, lo cual nos presenta la particularidad de tener columnas con valores que representan lo mismo pero están expresados de manera diferente. En esta ocasión, analizaremos específicamente la columna que contiene los valores categóricos de los canales de comercialización.

In [7]:
canales_unicos_de_tareas= df_tareas['Canal'].unique()
canales_unicos_de_entrada_salida= df_entrada_salida['Tipo de Punto de Venta'].unique()

print(f"los canales unicos del la columna canal del reporte de tareas son:{canales_unicos_de_tareas}")
print(f"los canales unicos del la columna Tipo de Punto de Venta del reporte de entrada y salida:{canales_unicos_de_entrada_salida}")

los canales unicos del la columna canal del reporte de tareas son:['Maxi Kiosco' 'Autoservicio / Chino' 'Kiosco Ventana' 'Almacén'
 'Verdulería' 'Dietetíca' 'Bar' 'Golsinería' 'Fiambrería']
los canales unicos del la columna Tipo de Punto de Venta del reporte de entrada y salida:['Maxi Kiosco' 'Almacén' 'Autoservicio' 'Goloteca' 'Verdulería' 'Bar'
 'Kiosco Ventana' 'Dietética' 'Golsinería' 'Farmacia / Perfumería'
 'Carnicería' 'Vinoteca']


In [8]:
df_tareas['Canal'] = df_tareas['Canal'].replace({'Autoservicio / Chino':'Autoservicio'})

Como se mencionó en el apartado de contexto y carga de datos, disponemos de tres tipos diferentes de reportes que contienen información complementaria. Esto nos permite observar con mayor detalle las distintas instancias generadas a través del censo de locales. Nuestro primer paso será comparar las dimensiones de las diferentes tablas con los valores únicos de la columna 'identificador de punto de interés' correspondiente a cada tipo de reporte. Esto nos proporcionará información sobre los registros duplicados.

In [9]:
# Dimensiones de los DataFrames
dimensiones_df_entrada_salida = df_entrada_salida.shape
dimensiones_df_observaciones = df_observaciones.shape
dimensiones_df_tareas = df_tareas.shape

# Cantidad de identificadores únicos en cada DataFrame
identificadores_unicos_entrada_salida = df_entrada_salida['Identificador de Punto de Interés'].nunique()
identificadores_unicos_observaciones = df_observaciones['Identificador Punto de interés'].nunique()
identificadores_unicos_tareas = df_tareas['Identificador del punto de interés'].nunique()

# resultados
print(f"Dimensiones de df_entrada_salida: {dimensiones_df_entrada_salida}")
print(f"Cantidad de identificadores únicos en df_entrada_salida: {identificadores_unicos_entrada_salida}")

print(f"Dimensiones de df_observaciones: {dimensiones_df_observaciones}")
print(f"Cantidad de identificadores únicos en df_observaciones: {identificadores_unicos_observaciones}")

print(f"Dimensiones de df_tareas: {dimensiones_df_tareas}")
print(f"Cantidad de identificadores únicos en df_tareas: {identificadores_unicos_tareas}")


Dimensiones de df_entrada_salida: (279, 18)
Cantidad de identificadores únicos en df_entrada_salida: 273
Dimensiones de df_observaciones: (29, 5)
Cantidad de identificadores únicos en df_observaciones: 28
Dimensiones de df_tareas: (184, 82)
Cantidad de identificadores únicos en df_tareas: 184


Como se puede observar en el resultado anterior, tenemos valores repetidos en el reporte de entrada y salida manual. Esto puede deberse a varios factores, entre ellos que el censista cargue información adicional en una instancia posterior que no fue incluida en la primera, haciendo que un registro complemente a otro. Para abordar esto, realizaremos un bucle que itere por las columnas, comparando si los registros son idénticos o diferentes. Esto nos permitirá examinar con más detalle la naturaleza de los valores repetidos y así generar conclusiones claras.

## **LIMPIEZA DE VALORES REPETIDOS**

Para abordar la limpieza de valores repetidos, combinaremos los registros donde el identificador de punto de interés esté duplicado y unificaremos los valores. Esto significa que si una columna X tiene un valor nulo para un identificador en un registro duplicado pero no en otro, conservaremos el registro que contenga información válida. Es importante destacar que este proceso eliminará algunos valores repetidos. Sin embargo, si encontramos registros duplicados donde una columna tiene valores válidos pero diferentes, se conservará dicho registro.

In [10]:
# Identificar filas duplicadas basadas en el identificador único
duplicados = df_entrada_salida[df_entrada_salida.duplicated(subset='Identificador de Punto de Interés', keep=False)]

# Ordenar por el identificador único para observar las duplicidades juntas
duplicados_ordenados = duplicados.sort_values(by='Identificador de Punto de Interés')

# Función para contar valores no nulos en una fila
def contar_no_nulos(row):
    return row.notnull().sum()

# Comparar filas duplicadas y quedarse con la que tenga más información no nula
def seleccionar_mejor_registro(df, id_col):
    mejores_registros = []
    unique_ids = df[id_col].unique()
    
    for unique_id in unique_ids:
        filas = df[df[id_col] == unique_id]
        if len(filas) > 1:
            filas['no_nulos'] = filas.apply(contar_no_nulos, axis=1)
            mejor_fila = filas.loc[filas['no_nulos'].idxmax()]
            mejores_registros.append(mejor_fila.drop('no_nulos'))
        else:
            mejores_registros.append(filas.iloc[0])
    
    return pd.DataFrame(mejores_registros)

# Seleccionar el mejor registro para cada identificador duplicado
df_mejores_registros = seleccionar_mejor_registro(df_entrada_salida, 'Identificador de Punto de Interés')

# Unir los registros únicos no duplicados con los mejores registros seleccionados
df_unicos = df_entrada_salida.drop_duplicates(subset='Identificador de Punto de Interés', keep=False)
df_entrada_salida = pd.concat([df_unicos, df_mejores_registros]).drop_duplicates(subset='Identificador de Punto de Interés')


C:\Users\sosaj\AppData\Local\Temp\ipykernel_16980\208835376.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filas['no_nulos'] = filas.apply(contar_no_nulos, axis=1)
C:\Users\sosaj\AppData\Local\Temp\ipykernel_16980\208835376.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filas['no_nulos'] = filas.apply(contar_no_nulos, axis=1)
C:\Users\sosaj\AppData\Local\Temp\ipykernel_16980\208835376.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

## **UNIFICACIÓN DE COLUMNAS**

Para unificar las columnas, procederemos con una concatenación de datos. Después de haber revisado la naturaleza de los registros en las diferentes tablas y haber eliminado los identificadores únicos del reporte de entrada y salida manual que también están presentes en los reportes de tareas y observaciones, nos aseguraremos de tener solo registros únicos. Una vez obtenidos estos registros únicos, procederemos con la concatenación.

Esta forma de consolidar tablas de datos difiere del uso de `left_on` o `merge`, que realizan uniones basadas en columnas. La concatenación une datos en una sola tabla. Si las columnas tienen el mismo nombre, se fusionan en una sola columna. Las columnas que están presentes en un registro pero no en otro se conservarán, llenando las celdas correspondientes con el valor "N/A" para los registros que no pertenecen a ese reporte.

In [11]:
# Paso 1: Obtener identificadores únicos de cada DataFrame
identificadores_observaciones = set(df_observaciones['Identificador Punto de interés'])
identificadores_tareas = set(df_tareas['Identificador del punto de interés'])
identificadores_entrada_salida = set(df_entrada_salida['Identificador de Punto de Interés'])

# Encontrar las coincidencias y no coincidencias entre df_tareas y df_observaciones
coincidencias = identificadores_tareas.intersection(identificadores_observaciones)
no_coincidencias_tareas = identificadores_tareas - identificadores_observaciones
no_coincidencias_observaciones = identificadores_observaciones - identificadores_tareas

# Crear DataFrames de coincidencias y no coincidencias
df_tareas_coincidencias = df_tareas[df_tareas['Identificador del punto de interés'].isin(coincidencias)]
df_observaciones_no_coincidencias = df_observaciones[df_observaciones['Identificador Punto de interés'].isin(no_coincidencias_observaciones)]
df_tareas_no_coincidencias = df_tareas[df_tareas['Identificador del punto de interés'].isin(no_coincidencias_tareas)]

# Unión de DataFrames
df_union = pd.concat([df_tareas_coincidencias, df_observaciones_no_coincidencias, df_tareas_no_coincidencias])

# Paso 2: Filtrar el DataFrame unido por canales deseados
canales_deseados = ["Almacén", "Autoservicio", "Goloteca", "Golsinería", "Kiosco Ventana", "Maxi Kiosco", "Autoservicio"]
df_union_filtrado = df_union[df_union['Canal'].isin(canales_deseados)]

# Paso 3: Filtrar df_entrada_salida por canales deseados
df_entrada_salida_filtrado = df_entrada_salida[df_entrada_salida['Jerarquía nivel 1'].isin(canales_deseados)]

# Paso 4: Comparar la cantidad de registros únicos entre los DataFrames resultantes
cantidad_unicos_union = df_union_filtrado['Identificador del punto de interés'].nunique()
cantidad_unicos_entrada_salida = df_entrada_salida_filtrado['Identificador de Punto de Interés'].nunique()

print(f"Cantidad de identificadores únicos en df_union_filtrado: {cantidad_unicos_union}")
print(f"Cantidad de identificadores únicos en df_entrada_salida_filtrado: {cantidad_unicos_entrada_salida}")

# Hipótesis: Verificar si ambos DataFrames tienen la misma cantidad de registros únicos
if cantidad_unicos_union == cantidad_unicos_entrada_salida:
    print("La hipótesis es correcta: ambos DataFrames tienen la misma cantidad de registros únicos.")
else:
    print("La hipótesis no es correcta: los DataFrames tienen diferente cantidad de registros únicos.")


Cantidad de identificadores únicos en df_union_filtrado: 171
Cantidad de identificadores únicos en df_entrada_salida_filtrado: 203
La hipótesis no es correcta: los DataFrames tienen diferente cantidad de registros únicos.


## **HIPÓTESIS**

Dada la cantidad de registros y la naturaleza variable de la recopilación de datos por parte de los censistas, es posible que se tomen ciertas libertades al ingresar información en algunos campos. Esto nos impide establecer un esquema estático para determinar qué registros son más adecuados. Nuestra estrategia consistirá en conservar todos los registros, utilizar aquellos que consideremos pertinentes para alimentar el tablero de control y luego realizar una limpieza exhaustiva para obtener el producto final.

## **CAMBIO DE NOMBRES DE COLUMNAS**

Realizaremos un cambio de nombres en nuestras columnas para unificarlas en la concatenación final.

In [12]:
# Renombrar columnas de df_observaciones
df_observaciones = df_observaciones.rename(columns={
    'Identificador Punto de interés': 'identificador_pdv',
    'Punto de interés': 'nombre_alta'
})

# Renombrar columnas de df_tareas
df_tareas = df_tareas.rename(columns={
    'Fecha de inicio': 'Fecha',
    'Identificador del punto de interés': 'identificador_pdv',
    'Nombre del punto de interés': 'nombre_alta',
    'Nombre de persona de interés': 'nombre_censista',
    'Coordenadas inicio': 'Coordenadas',
    'Identificador de persona de interés': 'identificado_censita'
})

# Renombrar columnas de df_entrada_salida
df_entrada_salida = df_entrada_salida.rename(columns={
    'Identificador Persona de interés': 'identificado_censita',
    'Nombre de Punto de interés': 'nombre_alta',
    'Fecha/Hora Entrada': 'Fecha',
    'Tipo de Punto de Venta': 'Canal',
    'Departamentos': 'Departamento',
    'Coordenadas de salida': 'Coordenadas',
    'Nombre de Persona de interés': 'nombre_censista',
    'Identificador de Punto de Interés':'identificador_pdv'
})


## **UNIÓN ENTRE TABLAS: REPORTE DE OBSERVACIONES Y REPORTE DE ENTRADA Y SALIDA MANUAL**

Primero procederemos a realizar una fusión entre la tabla perteneciente al reporte de observaciones y la tabla correspondiente al reporte de entrada y salida manual. Esto nos permitirá obtener información adicional de las observaciones, como el canal, coordenadas, etc.

In [13]:
df_observaciones = pd.merge(df_observaciones, df_entrada_salida, on='identificador_pdv', how='left')

In [14]:
df_observaciones.columns

Index(['Fecha_x', 'identificador_pdv', 'nombre_alta_x', 'Tipos de observación',
       'Descripción', 'identificado_censita', 'nombre_censista',
       'Número de celular', 'nombre_alta_y', 'Jerarquía nivel 1', 'Fecha_y',
       'Fecha/Hora Salida',
       '¿Realizó alguna acción? Llámese tareas, control de sku, observaciones, etc',
       'Coordenadas', 'Editado', 'Canal', 'Provincia', 'Departamento',
       'Localidad', 'Número de Polígono', 'Dirección2', 'Departamento3'],
      dtype='object')

In [15]:
# Elimino coplumnas repetidas
columnas_eliminar = ['identificado_censita', 'nombre_censista','nombre_alta_y', 'Fecha_y','Dirección2', 'Departamento3']
df_observaciones =  df_observaciones.drop(columns=columnas_eliminar)

In [16]:

# Renombrar columnas de df_observaciones
df_observaciones = df_observaciones.rename(columns={
    'Fecha_x': 'Fecha',
    'nombre_alta_x': 'nombre_alta'
})

## **REPORTE DE TAREAS**

A continuación, procederemos con la limpieza y preparación de la tabla correspondiente al reporte de tareas. En primer lugar, realizaremos un cambio en los nombres de las variables. Luego, unificaremos las columnas y llevaremos a cabo la limpieza necesaria. Finalmente, procederemos con la concatenación de los datos.

In [17]:
nuevos_nombres = {'Cantidad de Cajas': 'Cantidad de Cajas(Autoservicio)',
                  'Cantidad de Góndolas': 'Cantidad de Góndolas(Autoservicio)',
                  'Tiene Verdulería': 'Tiene Verdulería(Autoservicio)',
                  'Tiene Fiambrería': 'Tiene Fiambrería(Autoservicio)',
                  'Heladeras con lacteos': 'Heladeras con lacteos(Autoservicio)',
                  'Heladeras con bebidas': 'Heladeras con bebidas(Autoservicio)',
                  'Cantidad de Exhibidores de primeras marcas (Lays/Gaseosas/Cervezas)': 'Cantidad de Exhibidores de primeras marcas(Almacen)',
                  'Tiene surtido de vinos': 'Tiene surtido de vinos(Almacen)',
                  'Tiene Heladera de lácteos': 'Tiene Heladera de lácteos(Almacen)',
                  'Tiene gondola': 'Tiene gondola(Maxi Kiosco)',
                  'En caso de que si tenga gondola, cuan llena esta?': 'si tenga gondola, cuan llena esta?(Maxi Kiosco)',
                  'Tiene exhibidor de chicles/caramelos': 'Tiene exhibidor de chicles/caramelos(Maxi Kiosco)',
                  'En caso de tener exhibidor de chicles/caramelos, cuan lleno esta?': 'cuan lleno esta?(Maxi Kiosco)',
                  'No tiene servicios adicionales': 'No tiene servicios adicionales(Maxi Kiosco)',
                  'Fotocopias': 'Fotocopias(Maxi Kiosco)',
                  'RapiPago': 'RapiPago(Maxi Kiosco)',
                  'Tramites': 'Tramites(Maxi Kiosco)',
                  'SUBE': 'SUBE(Maxi Kiosco)',
                 'Cuan llena estan las estanterias': 'Cuan llena estan las estanterias(Kiosco Ventana)',
                 'Comercializa chocolates': 'Comercializa chocolates(Vinoteca)',
                  'Comercializa productos de kiosco': 'Comercializa productos de kiosco(farmacia)',
                  'Modalidad de Venta': 'Modalidad de Venta(Goloteca/Golsineria)',
                  'Tabletas': 'Tabletas(Goloteca/Golsineria)',
                  'Bombones': 'Bombones(Goloteca/Golsineria)',
                  'Alfajores': 'Alfajores(Goloteca/Golsineria)',
                  'Chicles': 'Chicles(Goloteca/Golsineria)',
                  'Caramelos': 'Caramelos(Goloteca/Golsineria)',
                  'Galletas': 'Galletas(Goloteca/Golsineria)',
                  'Cantidad de góndolas/estanterías': 'Cantidad de góndolas/estanterías(Dietetica)',
                 'BC': 'BC(Dietetica)',
                  'Aguila': 'Aguila(Dietetica)',
                  'CerealMix': 'CerealMix(Dietetica)',
                  'Turrón Arcor': 'Turrón Arcor(Dietetica)',
                  'Chocolate sin Azúcar Georgalos': 'Chocolate sin Azúcar Georgalos(Dietetica)',
                  'Llenado de estanterías': 'Llenado de estanterías(Dietetica)',
                  'Georgalos': 'georgalos_e',
                  'Cigarros': 'cigarillos_e',
                  'Identificador de persona de interés': 'ID_censista',
                  'Mondelez': 'MDLZ_1',
                  'Arcor': 'Arcor_1',
                  'Arcor8':'Arcor_e',
                  'Mondelez9':'mondelez_e',
                  'Coca Cola':'cocacola_e',
                  'Quilmes':'quilmes_e',
                  'Pepsi':'pepsi_e',
                  'Cigarrillos':'cigarillos_e',
                  'mkm ':'mkm',
                  'mkm ':'mkm',
                  'delez ':'delez'
                  
                  
                  }
df_tareas = df_tareas.rename(columns=nuevos_nombres)

In [18]:
nuevos_valores = {

"abc" : "ABC",
"ABC" : "ABC", 
"Abc distr" : "ABC",
"abc" : "ABC", 
"Abc. Marcelo" : "ABC",
"Norte y ABC" : "ABC",
"Abc" : "ABC",
"Norte y abc" : "ABC",
"Marcelo" : "ABC",
"Rafael abc" : "ABC",
"Andertich" : "Andertich",
"Andertrch" : "Andertich",
"Andertich" : "Andertich",
"Andertich" : "Andertich",
"Andertich" : "Andertich",
"Andertich" : "Andertich",
"Andertich" : "Andertich",
"Andertich" : "Andertich",
"Varios. Arcor, mundo dulce, el abasto, otros" : "Mundo Dulce y otros",
"Quilmes, coca, JB, planeta dulce, manaos. Además preventistas de mayoristas." : "Mundo Dulce y otros",
"Quilmes, coca cola, abirras, cervezas artesanales, jota be y mundo dulce." : "Mundo Dulce y otros",
"Quilmes, coca cola, mundo dulce, arcor, JB, Ramonda. Otros" : "Mundo Dulce y otros",
"Quilmes, coca cola, mundo dulce, otros": "Mundo Dulce y otros",
"Mundo dulce, arcor, quilmes, coca, manaos, silvia. Otros." : "Mundo Dulce y otros",
"Mundo dulce, quilmes" : "Mundo Dulce y otros",
"JB, Quilmes, coca, mundo dulce, manaos, otros" : "Mundo Dulce y otros",
"Coca, quilmes, logex, jota be, mundo dulce, arcor, manaos, fema, the king. Otros" : "Mundo Dulce y otros",
"Bebidas quilmes, coca, manaos, lacteos clarita, dulce mundo dulce y arcor, otros." : "Mundo Dulce y otros",
"Arcor, quilmes, mundo dulce, JB, otros": "Mundo Dulce y otros",
"Coca, quilmes, jota be, mundo dulce, otros" : "Mundo Dulce y otros",
"Coca, manaos, secco, quilmes, arcor, mundo dulce, jb, abirras, otros" : "Mundo Dulce y otros",
"Coca cola, quilmes, manaos, JB, mundo dulce, Ramonda, el abasto, varios" : "Mundo Dulce y otros",
"Mundo dulce, arcor, quilmes, coca, manaos, silvia. Otros." : "Mundo Dulce y otros",
"Producen panificados y la parte de autoservicio se abastecen por distribuidoras jota be, quilmes, silvia, coca cola, mundo dulce." : "Mundo Dulce y otros",
"Productores de productos regionales y distribuidor para gaseosas y dulces (mundo dulce)" : "Mundo Dulce y otros",
"Arcor dyssa" : "Dyssa",
"Dyssa" : "Dyssa",
"Dysaa arcor" : "Dyssa",
"dysa" : "Dyssa",
"Dysa" : "Dyssa",
"Dyssa Arcor" : "Dyssa",
"Dyssa arcor" : "Dyssa",
"Arcor dyssa copeti el reino" : "Dyssa",
"Arcor dissa" : "Dyssa",
"AyV López"  : "AyV López",
"Dankar.-" : "Dankar",
"Delez srl" : "Delez",
"Delez SRL" : "Delez",
"Delez" : "Delez",
"Delez Srl" : "Delez",
"Delez srk" : "Delez",
"Delez SRL" : "Delez",
"delez srl" : "Delez",
"Distribuidora de los Valles" : "Distribuidora de los Valles",
"Distribuidora de los valles" : "Distribuidora de los Valles", 
"Distribuidora de loa Valles" : "Distribuidora de los Valles", 
"El Reino" : "El Reino",
"el reino" : "El Reino",
"El reino" : "El Reino",
"Ek reino" : "El Reino",
"Gastricini" : "Gastricini", 
"Gastricinu" : "Gastricini",
"Gsstricini" : "Gastricini",
"Pepi distribuídor de Terrabusi" : "Global Terra", 
"Global terra" : "Global Terra", 
"Global Terra" : "Global Terra", 
"Globaltera" : "Global Terra", 
"Globalterra" : "Global Terra", 
"Globak terra" : "Global Terra", 
"Hard Sell" : "Hard Sell", 
"Hard sell" : "Hard Sell",
"Hard sell moreno" : "Hard Sell",
"Hard sell moreno." : "Hard Sell",
"Har Se" : "Hard Sell",
"Hatñrd sell" : "Hard Sell",
"Hard selll" : "Hard Sell",
"Har sell moreno" : "Hard Sell",
"Hand sell" : "Hard Sell",
"AyV López" :   "AyV López",                                                                                                                       
"AYV López" :   "AyV López",
"Ayv López" :   "AyV López",
"Mkm" : "MKM", 
"MKM" : "MKM", 
"Sandra" : "MKM",
"Mkm . Yanina" : "MKM", 
"Mkm terrabusi" : "MKM", 
"Mkm Terrabussi" : "MKM", 
"Mkm terrbusi" : "MKM", 
"Mkm terrabusj" : "MKM", 
"Yamila Mondelez" : "MKM", 
"mkm terrabusi" : "MKM", 
"Yamila" : "MKM", 
"Yanina" : "MKM",
"Angie" : "MKM",
"Muriel, nannis, sancor ._" : "Nani Hermanos",
"Nami hermanos" : "Nani Hermanos",
"Nani hermano" : "Nani Hermanos",
"NANI HERMANOS" : "Nani Hermanos",
"Nani hermanos" : "Nani Hermanos",
"Nani hnos" : "Nani Hermanos",
"Nani Hnos. Srl." : "Nani Hermanos",
"Nani hnos." : "Nani Hermanos",
"Nani hnos." : "Nani Hermanos",
"Nanni" : "Nani Hermanos", 
"Nanibhermanos" : "Nani Hermanos",
"Nanni, Dankar, Delorenzi, Cocacola Andina.-" : "Nani Hermanos",
"Nannis"  : "Nani Hermanos",
"Nany hermanos" : "Nani Hermanos",
"Nannis, muriel, delorenzi,dava,dizeo._" : "Nani Hermanos",
"Nannis, Muriel, Delorenzi,Coca cola andina.-" : "Nani Hermanos",
"Nannis.-" : "Nani Hermanos",
"Nannis, muriel, delorenzi,dava,dizeo._" : "Nani Hermanos",
"Nannis, Muriel, Delorenzi,Coca cola andina.-" : "Nani Hermanos",
"Nannis,Muriel,Delorenzi y Cocacola andina.-" : "Nani Hermanos",
"Nannis, Muriel,Delorenzi, Coca cola andina .-" : "Nani Hermanos",
"Nannis, De Lorenzi, Muriel, DiZeo y Cocacola andina.-" : "Nani Hermanos",
"Nannis, Delorenzi, Muriel, Moncho Benitez.-" : "Nani Hermanos",
"Nannis, Delorenzi, Muriel, Serenisima y Coca cola andina.-" : "Nani Hermanos",
"Nanni, Delorenzi, Cocacola andina, Muriel y Dankar.-" : "Nani Hermanos",
"Nany hermanos" : "Nani Hermanos",
"Nani hnos\r\n" : "Nani Hermanos",
"Nani hnos.\r\n" : "Nani Hermanos",
"Norte" : "Norte", 
"norte" : "Norte", 
"Pascual distribuciones" : "Pascual Distribuciones",                                                                                                            
"Pascual Distribuciones" : "Pascual Distribuciones", 
"Pareltein" : "Perelstein",
"Parelstein" : "Perelstein",
"Parelstain" : "Perelstein",
"Perelestien" : "Perelstein",
"Perenstein" : "Perelstein",
"Perensteind" : "Perelstein",
"Perelstein" : "Perelstein",
"Perlstein" : "Perelstein",
"Perlentein" : "Perelstein",
"Perlentein" : "Perelstein",
"Perlenstein" : "Perelstein",
"Peinsters" : "Perelstein",
"Perelesteind" : "Perelstein",
"Potegian" : "Potigian",
"Potigian" : "Potigian",
"Potigian (trabaja solo com YPF)" : "Potigian",
"Potigian" : "Potigian",
"Pianesola, el lealcito" : "Pianezolla", 
"Pianezola" : "Pianezolla", 
"Piabezolla" : "Pianezolla", 
"Pianesola" : "Pianezolla", 
"Pianezolla" : "Pianezolla", 
"Pinezolla" : "Pianezolla",
"QUI-PAR" : "QUI-PAR",
"Quipar" : "QUI-PAR",
"Distribuidores mas conocidos" : "Coca, Arcor, Pepsi y otros",  
"Casi todos (6)" : "Coca, Arcor, Pepsi y otros", 
"Arcor" : "Coca, Arcor, Pepsi y otros", 
"Arcot" : "Coca, Arcor, Pepsi y otros", 
"Arcor pepsico coca" : "Coca, Arcor, Pepsi y otros",
"Arcor entre otros" : "Coca, Arcor, Pepsi y otros",
"Arcor, quilmes, otros de productos alimenticios"  : "Coca, Arcor, Pepsi y otros", 
"Arcor, quilmes, coca cola, manaos, otros" : "Coca, Arcor, Pepsi y otros",
"Quilmes, arcor, coca cola, todos los grandes distribuidores" : "Coca, Arcor, Pepsi y otros",
"Beracca" : "Coca, Arcor, Pepsi y otros",
"Quilmes, arcor, coca, fema, unilever, nestle, levite, la serenisima,ilolay, entre otros" : "Coca, Arcor, Pepsi y otros",
"Burgos" : "Coca, Arcor, Pepsi y otros",
"Quilmes, coca cola, maxico, tregar, la serenisima, etc"  : "Coca, Arcor, Pepsi y otros",
"Quilmes, coca, arcor, manaos, jotabe" : "Coca, Arcor, Pepsi y otros",
"Quilmes, Jota be, coca cola, ramonda, mym, manaos, Aloha, arcor. Otros" : "Coca, Arcor, Pepsi y otros",
"Distribuidora Silvia, Quilmes, arcor,JotaBe, coca cola, manaos, mercado concentrador" : "Coca, Arcor, Pepsi y otros",
"Coca cola" : "Coca, Arcor, Pepsi y otros",
"Coca cola, arcor, pepsi y quilmez" : "Coca, Arcor, Pepsi y otros", 
"Grandes distribuidores (coca, jotabe, productores regionales)" : "Coca, Arcor, Pepsi y otros", 
"Dankar.-" : "Coca, Arcor, Pepsi y otros",
"Pepsi, coca cola, pepsico, codeboa, pianezola, secco" : "Coca, Arcor, Pepsi y otros",
 "Varios" : "Coca, Arcor, Pepsi y otros",
"Maxico" : "Coca, Arcor, Pepsi y otros",
"Pepsico" : "Coca, Arcor, Pepsi y otros",
"Varias" : "Coca, Arcor, Pepsi y otros",
"Abasto, quilmes, coca cola, kerps, otros" : "Coca, Arcor, Pepsi y otros",
"Arcor, pianezola, pepsi, coca, codenoa, secco, cenoa, etc." : "Coca, Arcor, Pepsi y otros",
"Distribuidor nacional" : "Coca, Arcor, Pepsi y otros",
"Fd" : "Coca, Arcor, Pepsi y otros",
"JB, quilmes, coca cola, ramonda, otros" : "Coca, Arcor, Pepsi y otros",
"Karamanef" : "Coca, Arcor, Pepsi y otros",
"Las mencionadas anteriormente.-" : "Coca, Arcor, Pepsi y otros",
"Los chicos" : "Coca, Arcor, Pepsi y otros",
"Los mencionados anteriormente mas Sancor e Ilolay.-" : "Coca, Arcor, Pepsi y otros",
"Los mencionados anteriormente.-" : "Coca, Arcor, Pepsi y otros",
"Los mencionados antrteriormente mas Ilolay , Sancor, Manaos y CCu.-" : "Coca, Arcor, Pepsi y otros",
"Los nombrados anteriormente" : "Coca, Arcor, Pepsi y otros",
"Mas de 7 distribuidores" : "Coca, Arcor, Pepsi y otros",
"Matsico" : "Coca, Arcor, Pepsi y otros",
"Matsico" : "Coca, Arcor, Pepsi y otros",
"Maxico, coca cola, quilmes, etc." : "Coca, Arcor, Pepsi y otros",
"JB, verdana, otros" : "Coca, Arcor, Pepsi y otros",
"Maxico, quilmes, coca cola, la serenisima" : "Coca, Arcor, Pepsi y otros",
"Preventista de diversas marcas" : "Coca, Arcor, Pepsi y otros",
"Preventistas de diversas marcas" : "Coca, Arcor, Pepsi y otros",
"Grandes distribuidores y preventistas de mayoristas" : "Coca, Arcor, Pepsi y otros",
"Todos los distribuidores grandes" : "Coca, Arcor, Pepsi y otros",
"Todos los anteriormente tildados en la app._" : "Coca, Arcor, Pepsi y otros",
"Los mencinados en el punto anterior.-" : "Coca, Arcor, Pepsi y otros",
"No se acuerda" : "No Saben", 
"No recuerda" : "No Saben",
"No me espesifico, pero varios son los que pasan" : "No Saben",
"No sabe" : "No Saben", 
"No recuerda el nombre" : "No Saben", 
"No recuerda nombre" : "No Saben", 
"Desconoce" : "No Saben", 
"No recuerda el nombre de la vendedora" : "No Saben", 
"Desconose" : "No Saben", 
"El empleado no sabe bien que distribuidor viene, el solo recive" : "No Saben", 
"Empleado no puede proveerme esa informacion" : "No Saben", 
"Familiar del comerciante desconoce distribuidor" : "No Saben", 
"No info" : "No Saben", 
"No me quiere decir" : "No Saben", 
"No me saben decir" : "No Saben", 
"No provee esa informacion" : "No Saben", 
"No puede proveerme esa informacion" : "No Saben", 
"No quiere proveerme esa informacion" : "No Saben", 
"No recuerda 'varios'" : "No Saben", 
"No recuerda cree que si" : "No Saben", 
"No recuerda el nombre de la chica que es nueva" : "No Saben", 
"No recuerda el nombre/ una chica" : "No Saben", 
"No recuerdo" : "No Saben", 
"No Sabe" : "No Saben", 
"No sabe el nombre" : "No Saben", 
"No sabe la empleada" : "No Saben", 
"No sabe precisar la informacion" : "No Saben", 
"No saben" : "No Saben", 
"No saben son empleados" : "No Saben", 
"No se acuerda la empleada" : "No Saben", 
"No se acuerdan" : "No Saben", 
"No dicr" : "No Saben", 
"Una chica nueva/ no sabe el nombre" : "No Saben", 
"Empleado deconoce el distri" : "No Saben", 
"Una chica / no recuerda el nombre" : "No Saben",
"Mondeliz" : "Mondeléz/Terrabusi (sin especificar)", 
"Momzelez" : "Mondeléz/Terrabusi (sin especificar)", 
"Arcor terrabusi" : "Mondeléz/Terrabusi (sin especificar)", 
"Terrabussi" : "Mondeléz/Terrabusi (sin especificar)", 
"Mondelez" : "Mondeléz/Terrabusi (sin especificar)", 
"Arcor terrabussi" : "Mondeléz/Terrabusi (sin especificar)", 
"Terrabusi" : "Mondeléz/Terrabusi (sin especificar)", 
"Terrabusi y otro sin especificar" : "Mondeléz/Terrabusi (sin especificar)", 
"Terrabusi," : "Mondeléz/Terrabusi (sin especificar)", 
"Modelez" : "Mondeléz/Terrabusi (sin especificar)", 
"Arcir terrabusi" : "Mondeléz/Terrabusi (sin especificar)", 
"Terrabusi, maxico, quilmes, coca cola" : "Mondeléz/Terrabusi (sin especificar)",
"Tiene 2 distribuidores, Terrabusi y otro sin especificar" : "Mondeléz/Terrabusi (sin especificar)",
"Terrabusi (Posiblemente sea global terra)" : "Mondeléz/Terrabusi (sin especificar)",
"Magali" : "Impreciso", 
"Marcela" : "Impreciso", 
"Maxi" : "Impreciso", 
"Ignacio arias" : "Impreciso",  
"En general" : "Impreciso", 
"Diego" : "Impreciso", 
"Daniel" : "Impreciso", 
"Cecilia" : "Impreciso",  
"Angie (dejo de veenir)" : "Impreciso",  
"..." : "Impreciso", 
"Recibe en general a todos los distribuidores, no lleva cuenta que empresa son" : "Impreciso",
"Sandra k50" : "Impreciso",
"Vendedor 120" : "Impreciso",
"Vario" : "Impreciso",
"Si" : "Impreciso",
"San esteban" : "Impreciso",
"Revendedor" : "Impreciso",
"San esteban" : "Impreciso",
"Propio" : "Impreciso",
"No especifica" : "Impreciso",
"Distribuidor mayorista" : "Impreciso",
"Distribuciones srl entre otros que no recuerda" : "Impreciso",
"Directos" : "Impreciso",
"Mayorista" : "Impreciso",
"Vea" : "Impreciso",
"X" : "Impreciso",
"x" : "Impreciso",
"Ramonda, ccu": "Impreciso",
"No esta pasando" : "Impreciso",
"Cambiaron el vendedor y no esta pasando" : "Impreciso",
"Cambiaron el vendedor y no esta viniendo" : "Impreciso",
'Mkm ( mondelez)':'Mkm mondelez',
'distribuidora byb' :'byb',
'no saben':'sin dato',
'terra':'global terra',
'no se acuerda la emplead':'sin dato',
'dele':'delez',
'terra ':'global terra',
'distribuciones byv':'byb',
'dele,':'delez',
'gastriconi':'gastricini',
'gasteicini':'gastricini',
'mkm ':'mkm',
'delez ':'delez',
'oficial ':'oficial',
'hard sell ':'hard sell',
'gasyricini':'gastricini',
'qui-par ':'qui-par',
'mondelez ':'mondelez',
'oficial ':'oficial',
'Hard selloreno':'Hard sell',
'sin info':'sin dato',
'terra':'global terra',
'a y b':'byb',
'delez':'mondelez',
'mondelez ':'mondelez',
'hard selloreno':'Hard sell',
'byv distribuciones':'byv',
'byv distribuciones ':'byv',
'mkm ':'mkm',
'gasyeicini':'gastricini',
'pritty ':'pritty',
'terra modelo':'global terra',
'distribuidora ':'distribuidora',
'ayv lópez ':'ayv lopez',
's.r.l':'srl',
'mkm ':'mkm',
'distribuidora dlv ':'distribuidora dlv',
'ayv lópez ':'ayv lópez',
'competencia ':'competencia',
'hard sell moreno ':'hard sell',
'hardsell ':'hard sell',
'hardsell':'hard sell',
'terrq':'global terra',
'gasticini':'gastricini',
'mkm\n':'mkm'


}

df_tareas['Distribuidor'] = df_tareas['Distribuidor'].replace(nuevos_valores)
# Verificar los valores únicos después del mapeo
valores_unicos = df_tareas['Distribuidor'].unique()
print(valores_unicos)

df_tareas['Distribuidor'] = df_tareas['Distribuidor'].str.lower()

[nan 'SCN' 'Mondelez, arcor'
 'Coca-Cola,arcor,georgalos, mondelez y pepsico'
 'Mondelez, arcor, raz, potigian, enercruz, lays, villavicencio, barracas, hector habanos'
 'Mondeléz/Terrabusi (sin especificar)' 'Mondelez, arcor, potigian'
 'Mondelez, potigian' 'Coca-Cola,arcor,, mondelez y pepsico' 'Scn, arcor'
 'Coca-Cola,, mondelez y pepsico' 'Scn, arcor, once kioskos'
 'Coca-Cola,arcor,terrabusi, mondelez y pepsico'
 'Coca-Cola,arcor,georgalos, Terrabusi, mondelez y pepsico'
 'Mondelez,arcor y georgalos' 'Scn, arcor, once kioskos, masivo'
 'Scn, golomax, potigian' 'Mondelez, arcor, raz' 'Scn'
 'Scn, golomax, once kioskos, raz '
 'Coca-Cola,arcor,georgalos, mondelez,quilmes y pepsico'
 'Mondelez, arcor, golomax, raz, once kioskos'
 'Coca-Cola,arcor, mondelez y pepsico'
 'Scn, arcor, golomax, once kioskos, snack brothers'
 'Scn, golomax, potigian, raz' 'Scn, arcor, dungals'
 'Scn, arcor, di campa, raz' 'Scn, arcor, potigian '
 'Scn, arcor, nobleza, once kioskos' 'Scn, arcor, villavicenc

In [19]:
Nuevos_valores = {
    'cordoba ':'cordoba',
    'cordoba ':'cordoba',
    'Sumalao':'sumalao',
    'Parque san Martin ':'parque san martin',
    'parque san martin ':'parque san martin',
    'San isidro':'san isidro',
    'San Martin ':'san martin',
    'merlo ':'merlo',
    'Córdoba ':'cordoba',
    'posadas ':'posadas',
    'La Rioja ':'la rioja',
    'Florida':'florida',
    'San Juan':'san juan',
    'Rivadavia':'rivadavia',
    'Villa Maipú':'villa maipu',
    'villa maipú ':'villa maipu',
    'villa Maipú ':'villa maipu',
    'libertad ':'libertad',
    'La Rioja':'la Rioja',
    'boulogne ':'boulogne',
    'villa lynch ':'villa lynch',
    'billinghurst ':'billinghurst',
    'barrio arcoíris ':'barrio arcoíris',
    'olivos ':'olivos',
    'ballester ':'ballester',
    'posadas ':'posadas',
    'moreno ':'moreno',
    'posadas ':'posadas',
    'paso del rey ':'paso del rey',
    'jose leon suarez ':'jose leon suarez',
    'rio primero ':'rio primero',
    'florida ':'florida',
    'banda ':'banda',
    'Moreno ':'moreno',
    'olivos':'olivos',
    'rio cuarto ':'rio cuarto',
    'caucete ':'caucete',
    'muñiz ':'muñiz',
    'bella vista ':'bella vista',
    'san miguel ':'san miguel',
    'la lucila ':'la lucila',
    'pontevedra ':'pontevedra',
    'mariano acosta ':'mariano acosta'
}

df_tareas['Localidad'] = df_tareas['Localidad'].replace(Nuevos_valores)

# Verificar los valores únicos después del mapeo
valores_unicos = df_tareas['Localidad'].unique()
print(valores_unicos)

['Microcentro' 'microcentro' 'Recoleta' 'Microcentro ' 'Recoleta '
 'Almagro' 'Retiro' 'microcentro ' 'florida' 'Florida ' 'Olivos' 'Olivos '
 'Villa martelli' 'Villa Martelli ']


In [20]:

columnas_unificar = ['Cantidad de heladeras','Cantidad de Heladeras3' , 'Cantidad de Heladeras visibles','Cantidad de heladeras6']
df_tareas[columnas_unificar] = df_tareas[columnas_unificar].replace('N/A', 0)
# Reemplazar '3 o más' por 3
df_tareas[columnas_unificar] = df_tareas[columnas_unificar].replace('3 o más', 3)
df_tareas[columnas_unificar] = df_tareas[columnas_unificar].fillna(0)
df_tareas[columnas_unificar] = df_tareas[columnas_unificar].astype(int)
df_tareas['Cantidad_de_heladeras_1'] = df_tareas[columnas_unificar].sum(axis=1)
df_tareas = df_tareas.drop(columnas_unificar, axis=1)

In [21]:

columnas_unificar = ['Estado de Inmueble','Estado de Inmueble2', 'Estado del inmueble','Estado de inmueble4','Estado del inmueble5','Estado de Inmueble7']
for columna in columnas_unificar:
    df_tareas[columna] = df_tareas[columna].replace('N/A', ' ').fillna(' ')

df_tareas['Estado_de_Inmueble_1'] = df_tareas[columnas_unificar].sum(axis=1)
df = df_tareas.drop(columnas_unificar, axis=1)

In [22]:
df_tareas['Estado_de_Inmueble_1'].unique()

array(['  Ordenado y Limpio   ', 'Ordenado y limpio     ',
       '   Ordenado y Limpio  ', '      ', ' Ordenado y Limpio    ',
       '   En condiciones aceptables  ', '  En condiciones aceptables   ',
       'En condiciones aceptables     ', ' En condiciones aceptables    ',
       '     Ordenado y Limpio', '   Precario  ', '  Precario   ',
       '    Ordenado y Limpio ', 'Precario     '], dtype=object)

In [23]:
remplazos={'   Ordenado y Limpio ':'Ordenado y Limpio',
    ' Ordenado y Limpio   ':'Ordenado y Limpio',
    'Ordenado y limpio    ':'Ordenado y Limpio',
    '  Ordenado y Limpio  ':'Ordenado y Limpio',
    '    Ordenado y Limpio':'Ordenado y Limpio',
    '   En condiciones aceptables ':'En condiciones aceptables',
    ' En condiciones aceptables   ':'En condiciones aceptables',
    '  En condiciones aceptables  ':'En condiciones aceptables',
    'En condiciones aceptables    ':'En condiciones aceptables',
    '    En condiciones aceptables':'En condiciones aceptables',
    '   En condiciones aceptables ':'En condiciones aceptables',
    '   En condiciones aceptables ':'En condiciones aceptables',
    '   Precario ':'Precario',
    ' Precario   ':'Precario',
    'Precario    ':'Precario',
    '    Precario':'Precario',
    '  Precario  ':'Precario',   
    'Ordenado y Limpio    ':'Ordenado y Limpio',
    '    Ordenado y limpio':'Ordenado y limpio',
    'Ordenado y limpio':'Ordenado y Limpio',
    '   En condiciones aceptables   ':'En condiciones aceptables',
    'Ordenado y Limpio      ':'Ordenado y Limpio',
    '     Ordenado y Limpio ':'Ordenado y Limpio',
    'Ordenado y Limpio ':'Ordenado y Limpio',
    '      Ordenado y Limpio':'Ordenado y Limpio',
    'En condiciones aceptables      ':'En condiciones aceptables',
    '      En condiciones aceptables':'En condiciones aceptables',
    '     En condiciones aceptables ':'En condiciones aceptables',
    ' En condiciones aceptables  En condiciones aceptables  ':'En condiciones aceptables',
    ' Ordenado y limpio  Ordenado y limpio  ':'Ordenado y Limpio',
    '   Ordenado y Limpio   ':'Ordenado y Limpio',
    '      Precario':'Precario',
    'Precario      ':'Precario',
    '  Ordenado y Limpio    ':'Ordenado y Limpio',
    ' Precario  Precario  ':'Precario',
    '     Precario ':'Precario',
    'Precario ':'Precario',
    '   Ordenado y Limpio  ':'Ordenado y Limpio',
    '     En condiciones aceptables':'En condiciones aceptables',
    '   En condiciones aceptables  ':'En condiciones aceptables',
    '     Ordenado y Limpio':'Ordenado y Limpio',
    '  Ordenado y Limpio   ':'Ordenado y Limpio',
    '  En condiciones aceptables   ':'En condiciones aceptables',
    '    Ordenado y limpio ':'Ordenado y limpio',
    ' Ordenado y Limpio    ':'Ordenado y Limpio',
    '     Precario':'Precario',
    'Ordenado y Limpio     ':'Ordenado y Limpio',
    ' En condiciones aceptables    ':'En condiciones aceptables',
    '    En condiciones aceptables ':'En condiciones aceptables',
    '   Precario  ':'Precario',
    '  Precario   ':'Precario',
    '    Precario ':'Precario',
    'En condiciones aceptables     ':'En condiciones aceptables',
    'Precario     ':'Precario',
    ' Precario    ':'Precario',
    'Ordenado y limpio':'Ordenado y Limpio',
    'Ordenado y limpio':'Ordenado y Limpio',
    'Ordenado y limpio':'Ordenado y Limpio',
    '    Ordenado y Limpio ':'Ordenado y Limpio',
    'Ordenado y limpio     ':'Ordenado y Limpio'
}

    # Aplicar el mapeo a la columna 'Localidad'
df_tareas['Estado_de_Inmueble_1'] = df_tareas['Estado_de_Inmueble_1'].replace(remplazos)

# Verificar los valores únicos después del mapeo
valores_unicos = df_tareas['Estado_de_Inmueble_1'].unique()
print(valores_unicos)

['Ordenado y Limpio' '      ' 'En condiciones aceptables' 'Precario']


In [24]:
import re

#  lista de palabras clave
palabras_clave = [ 'visita', 'preventista', 'mondelez', 'vendedor', 'delez','interesa']

# Función para buscar patrones en una observación
def buscar_patrones(observacion):
    if isinstance(observacion, str):
        observacion = observacion.lower()  # Convertir la observación a minúsculas
        for palabra in palabras_clave:
            patron = r'\b' + palabra + r'\w*'  # Usamos \w* para permitir cualquier carácter después de la palabra clave
            if re.search(patron, observacion):
                return 1
    return 0

# Aplicar la función a la columna "Observaciones" y crear la columna "Oportunidades"
df_tareas['Oportunidades'] = df_tareas['Observaciones'].apply(buscar_patrones)

In [25]:

# Eliminar de df_entrada_salida los identificadores que están en df_observaciones y df_tareas
ids_observaciones = set(df_observaciones['identificador_pdv'])
ids_tareas = set(df_tareas['identificador_pdv'])

# Crear un set con todos los ids que se deben eliminar de df_entrada_salida
ids_a_eliminar = ids_observaciones.union(ids_tareas)

# Filtrar df_entrada_salida
df_entrada_salida_filtrado = df_entrada_salida[~df_entrada_salida['identificador_pdv'].isin(ids_a_eliminar)]
df_entrada_salida_filtrado['origen'] = 'alta'

# Filtrar df_observaciones eliminando los identificadores que están en df_tareas
df_observaciones_filtrado = df_observaciones[~df_observaciones['identificador_pdv'].isin(ids_tareas)]
df_observaciones_filtrado['origen'] = 'observado'

# Agregar columna 'origen' a df_tareas
df_tareas['origen'] = 'efectivamente censado'

# Concatenar los DataFrames resultantes
df_concatenado = pd.concat([df_entrada_salida_filtrado, df_tareas, df_observaciones_filtrado], ignore_index=True)

# Mostrar el DataFrame resultante
print("DataFrame concatenado:")
print(df_concatenado.head())


DataFrame concatenado:
  identificado_censita       nombre_censista  Número de celular  \
0            TR-029883    Tobias  Rodríguez        1.159112e+09   
1            TR-029883    Tobias  Rodríguez        1.159112e+09   
2            RZ-623877  Ricardo Zorkendorfer       1.159112e+09   
3            RZ-623877  Ricardo Zorkendorfer       1.159112e+09   
4            TA-820118        Tomás Aquilino       1.159112e+09   

  identificador_pdv     nombre_alta Jerarquía nivel 1             Fecha  \
0        365-669675             365     Kiosco Cadena  02/05/2024 08:53   
1        365-047349             365     Kiosco Cadena  02/05/2024 09:21   
2        365-679929             365     Kiosco Cadena  02/05/2024 10:03   
3        365-117513             365     Kiosco Cadena  02/05/2024 15:54   
4        CLE-005443  Club saludable         Dietética  03/05/2024 15:57   

  Fecha/Hora Salida  \
0  02/05/2024 08:53   
1  02/05/2024 09:21   
2  02/05/2024 10:03   
3  02/05/2024 15:54   
4  03/05

C:\Users\sosaj\AppData\Local\Temp\ipykernel_16980\3035690207.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrada_salida_filtrado['origen'] = 'alta'


In [26]:
df_concatenado

,identificado_censita,nombre_censista,Número de celular,identificador_pdv,nombre_alta,Jerarquía nivel 1,Fecha,Fecha/Hora Salida,"¿Realizó alguna acción? Llámese tareas, control de sku, observaciones, etc",Coordenadas,Editado,Canal,Provincia,Departamento,Localidad,Número de Polígono,Dirección2,Departamento3,origen,Hora inicio,Fecha completado,Hora completado,Coordenadas fin,Número de teléfono,Jerarquia 1 del punto de interés,Departamentos,Tipo de Punto de Venta,Razón Social,CUIT,Papeles?,Cantidad de Cajas(Autoservicio),Cantidad de Góndolas(Autoservicio),Tiene Verdulería(Autoservicio),Tiene Fiambrería(Autoservicio),Heladeras con lacteos(Autoservicio),Heladeras con bebidas(Autoservicio),Estado de Inmueble,Cantidad de Exhibidores de primeras marcas(Almacen),Tiene surtido de vinos(Almacen),Tiene Heladera de lácteos(Almacen),Estado de Inmueble2,Estado del inmueble,Tiene gondola(Maxi Kiosco),"si tenga gondola, cuan llena esta?(Maxi Kiosco)",Tiene exhibidor de chicles/caramelos(Maxi Kiosco),cuan lleno esta?(Maxi Kiosco),No tiene servicios adicionales(Maxi Kiosco),Fotocopias(Maxi Kiosco),RapiPago(Maxi Kiosco),Tramites(Maxi Kiosco),SUBE(Maxi Kiosco),Cuan llena estan las estanterias(Kiosco Ventana),Estado de inmueble4,Comercializa chocolates(Vinoteca),Comercializa productos de kiosco(farmacia),Modalidad de Venta(Goloteca/Golsineria),Tabletas(Goloteca/Golsineria),Bombones(Goloteca/Golsineria),Alfajores(Goloteca/Golsineria),Chicles(Goloteca/Golsineria),Caramelos(Goloteca/Golsineria),Galletas(Goloteca/Golsineria),Estado del inmueble5,Cantidad de góndolas/estanterías(Dietetica),Llenado de estanterías(Dietetica),Estado de Inmueble7,Trabaja con?,Distribuidor,Frecuencia de Visita,Frecuencia de entrega,Se acerca un repositor de Mondelez?,Se acerca un repositor de la competencia?,"En caso de que sea otro, indicar con quien",Aplicaciones B2B,MDLZ_1,Arcor_1,BEES,Serenisima,Massalin,Arcor_e,georgalos_e,quilmes_e,pepsi_e,cocacola_e,cigarillos_e,mondelez_e,Observaciones,Cantidad_de_heladeras_1,Estado_de_Inmueble_1,Oportunidades,Tipos de observación,Descripción
0,TR-029883,Tobias Rodríguez,1.159112e+09,365-669675,365,Kiosco Cadena,02/05/2024 08:53,02/05/2024 08:53,No,"-34.58600190000000000000,-58.39867120000000000000",False,Maxi Kiosco,Buenos Aires,Capital,recoleta,NaN,NaN,NaN,alta,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TR-029883,Tobias Rodríguez,1.159112e+09,365-047349,365,Kiosco Cadena,02/05/2024 09:21,02/05/2024 09:21,No,"-34.58819660000000000000,-58.39038180000000000000",False,Maxi Kiosco,Buenos Aires,Capital,Recoleta,NaN,NaN,NaN,alta,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RZ-623877,Ricardo Zorkendorfer,1.159112e+09,365-679929,365,Kiosco Cadena,02/05/2024 10:03,02/05/2024 10:03,No,"-34.60189890000000000000,-58.38080660000000000000",False,Maxi Kiosco,Buenos Aires,Capital,Microcentro,NaN,NaN,NaN,alta,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RZ-623877,Ricardo Zorkendorfer,1.159112e+09,365-117513,365,Kiosco Cadena,02/05/2024 15:54,02/05/2024 15:54,No,"-34.60374140000000000000,-58.38367540000000000000",False,Maxi Kiosco,Buenos Aires,Capital,Microcentro,NaN,NaN,NaN,alta,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## **INTEGRIDAD DE LOS DATOS**

Para verificar la integridad de la concatenación, procederemos de la siguiente manera: primero, realizaremos un conteo de los valores categóricos de la columna 'Origen' en el dataframe concatenado. Luego, contaremos la cantidad de registros de cada tabla que se ha concatenado. Si la concatenación se ha realizado sin pérdida de datos, esperamos que los registros tengan las mismas dimensiones.

In [27]:
print(df_concatenado['origen'].value_counts())
print(df_tareas.shape)
print(df_entrada_salida_filtrado.shape)
print(df_observaciones.shape)

origen
efectivamente censado    184
alta                      61
observado                 29
Name: count, dtype: int64
(184, 82)
(61, 19)
(29, 16)


## **CONCLUSIONES FINALES**

A lo largo de este análisis, hemos identificado varias incongruencias respecto a la lógica esperada en el llenado de los campos por parte de los censistas y los resultados finales en la tabla consolidada. Uno de estos errores se evidencia en el siguiente caso:

Al tomar los identificadores únicos de los comercios del reporte de tareas y del reporte de observaciones, y eliminar estos identificadores del reporte de entrada y salida, al analizar el residuo encontramos canales de comercialización principales que no están presentes en el reporte de tareas. Esto indica que estos locales no fueron censados, pero tampoco tienen una razón documentada en el reporte de observaciones que explique su ausencia. Al revisar estos registros en detalle, notamos que el censista podría haber llenado algunas columnas de manera diferente, lo que podría explicar por qué no se registró adecuadamente este local en el reporte de tareas. 

In [28]:
valores_filtrar = ['Maxi Kiosco', 'Goloteca']
df_filtrado = df_entrada_salida_filtrado[df_entrada_salida_filtrado['Jerarquía nivel 1'].isin(valores_filtrar)]
from IPython.display import display, HTML
# Función para aplicar estilos condicionales
def highlight_canal(row):
    if row['Canal'] == 'Maxi Kiosco':
        return ['background-color: red'] * len(row)
    elif row['Canal'] == 'Goloteca':
        return ['background-color: blue'] * len(row)
    else:
        return [''] * len(row)

styled_df = df_filtrado.style.apply(highlight_canal, axis=1)
styled_df

,identificado_censita,nombre_censista,Número de celular,identificador_pdv,nombre_alta,Jerarquía nivel 1,Fecha,Fecha/Hora Salida,"¿Realizó alguna acción? Llámese tareas, control de sku, observaciones, etc",Coordenadas,Editado,Canal,Provincia,Departamento,Localidad,Número de Polígono,Dirección2,Departamento3,origen
32,RZ-623877,Ricardo Zorkendorfer,1159111825,COS-170060,Compañía de golosinas,Goloteca,02/05/2024 09:41,02/05/2024 09:41,No,"-34.60407740000000000000,-58.38297570000000000000",False,Goloteca,Buenos Aires,Capital,Microcentro,nan,nan,nan,alta
33,RZ-623877,Ricardo Zorkendorfer,1159111825,COS-163201,Compañía de golosinas,Goloteca,02/05/2024 14:05,02/05/2024 14:05,No,"-34.60158030000000000000,-58.38095040000000000000",False,Goloteca,Buenos Aires,Capital,Microcentro,nan,nan,nan,alta
34,RZ-623877,Ricardo Zorkendorfer,1159111825,COS-234936,Compañía de golosinas,Goloteca,02/05/2024 14:06,02/05/2024 14:06,No,"-34.60158030000000000000,-58.38095040000000000000",False,Goloteca,Buenos Aires,Capital,Microcentro,nan,nan,nan,alta
35,RZ-623877,Ricardo Zorkendorfer,1159111825,COS-302870,Compañía de golosinas,Goloteca,03/05/2024 16:45,03/05/2024 16:45,No,"-34.60362090000000000000,-58.37988810000000000000",False,Goloteca,Buenos Aires,Capital,Microcentro,nan,nan,nan,alta
36,TA-820118,Tomás Aquilino,1159111811,CO -661878,Compañía de golosinas,Goloteca,03/05/2024 16:09,03/05/2024 16:09,No,"-34.59974330000000000000,-58.39561200000000000000",False,Goloteca,Buenos Aires,Capital,Recoleta,nan,nan,nan,alta
79,TA-820118,Tomás Aquilino,1159111811,GO3-349490,Goloteca Pueyrredón 1033,Goloteca,03/05/2024 11:50,03/05/2024 11:50,No,"-34.59819920000000000000,-58.40382400000000000000",False,Goloteca,Buenos Aires,Capital,Recoleta,nan,nan,nan,alta
93,TR-029883,Tobias Rodríguez,1159111806,KIO-983886,Kiosco,Maxi Kiosco,03/05/2024 09:42,nan,No,"-34.60710670000000000000,-58.37564590000000000000",False,Maxi Kiosco,Buenos Aires,Capital,Microcentro,nan,nan,nan,alta
144,RZ-623877,Ricardo Zorkendorfer,1159111825,KIO-590670,Kiosko,Maxi Kiosco,03/05/2024 16:04,03/05/2024 16:04,No,"-34.60333760000000000000,-58.37475720000000000000",False,Maxi Kiosco,Buenos Aires,Capital,Microcentro,nan,nan,nan,alta


Como se puede observar en la tabla superior, tenemos la información de que se trata de una compañía de golosinas en el campo correspondiente a `nombre_alta`. Sin embargo, observamos que hay un registro donde el canal de comercialización es Maxi Kiosco, pero no está especificado en ningún campo si se trata de una cadena de Maxi Kioscos. Además, este registro no se encuentra ni en el reporte de tareas ni en el reporte de observaciones, por lo que a simple vista no se observa un motivo por el cual este local no haya sido censado con sus preguntas correspondientes.

Dada esta variabilidad en el llenado por parte de los censistas, los campos están expuestos a errores e inconsistencias. Por lo tanto, se sugiere construir una base de datos que alimente el tablero de Power BI, permitiendo observar los datos detalladamente con el tiempo y realizar la limpieza necesaria. De esta manera, al final del proceso se podrá entregar una tabla funcional para el análisis y la toma de decisiones.